In [10]:
from src.datasets.text_datasets.RestaurantDataset import RestaurantDataset
from src.datasets.text_datasets.AmazonDataset import AmazonDataset
from src.datasets.text_datasets.POIDataset import POIDataset

dataset = "restaurants".lower().replace(" ", "") 
subset = "gijon".lower().replace(" ", "") 

seed = 100 
l_rate = 1e-4
n_epochs = 1000 
b_size = 256 
early_stop_patience = 10

min_reviews_rst = 100
min_reviews_usr = 1
bow_pct_words = 10 
language = "es" if subset in ["gijon", "madrid", "barcelona"] else "fr" if subset in ["paris"] else "en"

remove_stopwords = 2  # 0, 1 o 2 (No quitar, quitar manual, quitar automático)
lemmatization = True
remove_accents = True
remove_numbers = True
truncate_padding = True

if dataset == "restaurants":
    base_path = "/media/nas/datasets/tripadvisor/restaurants/"
elif dataset == "pois":
    base_path = "/media/nas/datasets/tripadvisor/pois/"
    language = "es"  # Están todas en español
elif dataset == "amazon":
    base_path = "/media/nas/datasets/amazon/"

# DATASET CONFIG #######################################################################################################

dts_cfg = {"dataset": dataset, "subset": subset, "language": language, "seed": seed, "data_path": base_path, "save_path": "data/",  # base_path + "Datasets/",
            "remove_stopwords": remove_stopwords, "remove_accents": remove_accents, "remove_numbers": remove_numbers,
            "lemmatization": lemmatization,
            "min_reviews_rst": min_reviews_rst, "min_reviews_usr": min_reviews_usr,
            "min_df": 5, "bow_pct_words": bow_pct_words, "presencia": False, "text_column": "text",  # BOW
            "n_max_words": -50, "test_dev_split": .1, "truncate_padding": truncate_padding}

if dataset == "restaurants": text_dataset = RestaurantDataset(dts_cfg)
elif dataset == "pois": text_dataset = POIDataset(dts_cfg)
elif dataset == "amazon": text_dataset = AmazonDataset(dts_cfg)
else: raise ValueError

/media/nas/pperez/miniconda3/envs/TAVtext/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import gensim
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer

In [2]:
# Supongamos que tienes una lista de oraciones como datos de entrenamiento
sentences = [
    "El gato está en el tejado",
    "El perro está corriendo en el parque",
    "Los pájaros vuelan en el cielo"
]

# Tokenizar las oraciones en palabras individuales
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
tokenized_sentences = tokenizer.texts_to_sequences(sentences)

In [8]:
class TrainingCallback(gensim.models.callbacks.CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        print(f"Epoch {self.epoch:03d} => Loss:", loss)
        self.epoch+=1

In [18]:
tokenized_sentences = text_dataset.DATA["TEXT_SEQUENCES"]

# Definir los parámetros del modelo
embedding_dim = 100  # Dimensión del espacio de vectores
window_size = 5      # Tamaño de la ventana de contexto
epochs = 100         # Número de épocas de entrenamiento

# Entrenar el modelo Word2Vec
w2v_model = Word2Vec(sentences=tokenized_sentences, vector_size=embedding_dim, window=window_size, epochs=epochs, callbacks=[TrainingCallback()])

Exception in thread Thread-1204:
Traceback (most recent call last):
  File "/media/nas/pperez/miniconda3/envs/TAVtext/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-1205:
Traceback (most recent call last):
  File "/media/nas/pperez/miniconda3/envs/TAVtext/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-1206:
Traceback (most recent call last):
  File "/media/nas/pperez/miniconda3/envs/TAVtext/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/media/nas/pperez/miniconda3/envs/TAVtext/lib/python3.8/threading.py", line 870, in run
    self.run()
  File "/media/nas/pperez/miniconda3/envs/TAVtext/lib/python3.8/threading.py", line 870, in run
    self.run()
  File "/media/nas/pperez/miniconda3/envs/TAVtext/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/media/nas/pperez/miniconda3/envs/TAVtext/lib/python3.8/site-packages/gensim/model